# Prototype for splitting screen in block of pixel to reduce action spaces

In [ ]:
import numpy as np
import pandas as pd
import math

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import matplotlib.patches as patches

# Load images

In [ ]:
world_img = mpimg.imread('../../data/capture/world.png')
city_img = mpimg.imread('../../data/capture/city.png')
land_img = mpimg.imread('../../data/capture/land.png')

In [ ]:
height = 480
width = 640

In [ ]:
fig, ax = plt.subplots(1, 3, figsize = (25,10))

ax[0].imshow(world_img)
ax[1].imshow(city_img)
ax[2].imshow(land_img)
plt.show()

# Show different grid size

In [ ]:
def display_grid(axes, size: int):
    w = math.ceil(width / size)
    h = math.ceil(height / size)
    print(f'square of {size} : ({w} * {h}) = {w*h} actions')

    for pt in np.arange(0, width, size):
        for ax in axes:
            ax.axvline(pt, lw=0.5, color='black', alpha=1)

    for pt in np.arange(0, height, size):
        for ax in axes:
            ax.axhline(pt, lw=0.5, color='black', alpha=1)

In [ ]:
fig, ax = plt.subplots(1, 3, figsize = (25,10))

ax[0].imshow(world_img)
ax[1].imshow(city_img)
ax[2].imshow(land_img)

display_grid(ax, 10)
plt.show()

In [ ]:
fig, ax = plt.subplots(1, 3, figsize = (25,10))

ax[0].imshow(world_img)
ax[1].imshow(city_img)
ax[2].imshow(land_img)

display_grid(ax, 5)
plt.show()

# Converting Action to coordinate or coordinate to Action

In [ ]:
grid_size = 10
grid_width = math.ceil(width / grid_size)
grid_height = math.ceil(height / grid_size)
action_space = grid_width * grid_height

In [ ]:
def grid_to_pixel(x) -> float:
    return (x*grid_size) - (grid_size/2)

def display_click(ax, x, y):
    ax.plot(grid_to_pixel(x), grid_to_pixel(y), 'ro')

def display_drag(ax, x1, y1, x2, y2):
    # display_click(ax, x1, y1)

    w = x2 - x1
    h = y2 - y1

    ax.plot(grid_to_pixel(x2), grid_to_pixel(y2), 'yo')
    rect = patches.Rectangle((grid_to_pixel(x1), grid_to_pixel(y1)), w*grid_size, h*grid_size, linewidth=1, edgecolor='y', facecolor='none')
    ax.add_patch(rect)

### Display Click and Drag

In [ ]:
fig, ax = plt.subplots(1, figsize = (12,8))

ax.imshow(city_img)

display_grid([ax], grid_size)

# One click
display_click(ax, 40, 36)

# Zone selection
display_drag(ax, 24, 25, 26, 28)

plt.show()

### Action to coordinate - Click

In [ ]:
def play(ax, action: int):
    if action < 0 or action > action_space:
        raise ValueError(f'Action should be in range of action space [0, {action_space}]')
    
    x = action % grid_width + 1
    y = action // grid_width + 1

    print(f'action {action} : click at ({x}, {y})')
    display_click(ax, x, y)    


In [ ]:
fig, ax = plt.subplots(1, figsize = (12,8))

ax.imshow(city_img)

display_grid([ax], grid_size)

# Play multiple click to ensure they all end at right place
play(ax, 0)
play(ax, 1)
play(ax, 5)
play(ax, 9)
play(ax, 64*2)
play(ax, 500)
play(ax, 3071)

plt.show()

### Action to coordinate - Drag

In [ ]:
fig, ax = plt.subplots(1, figsize = (12,8))

ax.imshow(city_img)

display_grid([ax], grid_size)

last_click_x = 0
last_click_y = 0
def play2(ax, action: int):
    global last_click_x
    global last_click_y

    if action < 0 or action > action_space*2:
        raise ValueError(f'Action should be in range of action space [0, {action_space*2}]')

    c = action // action_space + 1
    x = (action%action_space) % grid_width + 1
    y = (action%action_space) // grid_width + 1

    if c == 1:
        print(f'action {action} : click at ({x}, {y})')
        display_click(ax, x, y)    

        last_click_x = x
        last_click_y = y
    elif c == 2:
        print(f'action {action} : drag from ({last_click_x}, {last_click_y}) to ({x}, {y})')
        display_drag(ax, last_click_x, last_click_y, x, y)    
    else:
        raise ValueError('Click type {c} is not supported')

def coordinate_to_action(x, y, click) -> int:
    return (x-1) + grid_width * (y-1) + ((click-1) * action_space)

# play(ax, 0 + action_space)

# play2(ax, 0)
play2(ax, coordinate_to_action(1, 1, 1))
play2(ax, coordinate_to_action(3, 2, 2))

# 24, 25, 26, 28
play2(ax, coordinate_to_action(24, 25, 1))
play2(ax, coordinate_to_action(26, 28, 2))

# play(ax, 64*2 + action_space)
# play(ax, 500 + action_space)
# play(ax, 3071 + action_space)

plt.show()